In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
from torchvision.models.detection.backbone_utils import resnet_fpn_backbone

In [ ]:
import torch
import torchvision
import torchvision.transforms as T

# Define a transform to preprocess the images
transform = T.Compose([
    T.Resize((800, 800)),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load the dataset
dataset = torchvision.datasets.ImageFolder('wm', transform=transform)

# Define the feature extractor
backbone = torchvision.models.detection.backbone_utils.resnet_fpn_backbone('resnet50', pretrained=True)

# Extract features for each image and save to a file
features = []
for image, label in dataset:
    # Preprocess the image
    image = image.unsqueeze(0)


    # Extract features
    with torch.no_grad():
        features.append(backbone(image)['0'].squeeze().cpu().numpy())

# Save the features to a file
features = np.array(features)
np.save('malignantfeatures.npy', features)


In [ ]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
import joblib
# Load the numpy files
features_class1 = np.load('benignfeatures.npy') # shape: (num_samples_class1, height, width, num_channels)
features_class2 = np.load('malignantfeatures.npy') # shape: (num_samples_class2, height, width, num_channels)

# Flatten or reshape the numpy arrays
X = np.concatenate((features_class1, features_class2), axis=0) # shape: (num_samples_class1 + num_samples_class2, height, width, num_channels)
X = np.reshape(X, (X.shape[0], -1)) # shape: (num_samples_class1 + num_samples_class2, num_features)

# Create the labels array
y = np.concatenate((np.zeros(features_class1.shape[0]), np.ones(features_class2.shape[0])))
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the logistic regression classifier on the training data

clf = DecisionTreeClassifier(random_state=0).fit(X_train, y_train)


# Evaluate the accuracy on the testing data
accuracy = clf.score(X_test, y_test)
print('Accuracy:', accuracy)

# Train the logistic regression classifier


# Save the classifier for future use
joblib.dump(clf, 'classifier.pkl')




Accuracy: 0.6666666666666666


['classifier.pkl']

In [ ]:
from keras.models import load_model

model = load_model('model.h5')
X_unlabeled = np.load('unlabeledfeatures.npy')
X_unlabeled = np.reshape(X_unlabeled, (X_unlabeled.shape[0], -1))
y_pred = model.predict(X_unlabeled)
print(y_pred)

1/1 [==============================] - 1s 1s/step
[[1.]]


In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
import joblib

# Load the numpy files
features_class1 = np.load('benignfeatures.npy') # shape: (num_samples_class1, height, width, num_channels)
features_class2 = np.load('malignantfeatures.npy') # shape: (num_samples_class2, height, width, num_channels)

# Flatten or reshape the numpy arrays
X = np.concatenate((features_class1, features_class2), axis=0) # shape: (num_samples_class1 + num_samples_class2, height, width, num_channels)
X = np.reshape(X, (X.shape[0], -1)) # shape: (num_samples_class1 + num_samples_class2, num_features)

# Create the labels array
y = np.concatenate((np.zeros(features_class1.shape[0]), np.ones(features_class2.shape[0])))

from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the neural network model
model = Sequential()
model.add(Dense(64, input_dim=X.shape[1], activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the neural network model on the training data
model.fit(X_train, y_train, epochs=10, batch_size=32)

# Evaluate the accuracy on the testing data
accuracy = model.evaluate(X_test, y_test)[1]
print('Accuracy:', accuracy)

# Save the model for future use
model.save('model.h5')
